In [202]:
import numpy as np
import pytreenet as ptn
from copy import deepcopy
np.random.seed(82)
X , Y , Z = ptn.pauli_matrices()
from pytreenet.contractions.state_state_contraction import contract_leafs , contract_all_but_one_neighbour_block_to_ket , contract_bra_to_ket_and_blocks_ignore_one_leg

In [203]:
def contract_leafs_2(node_id: str,
                   state1,
                   state2) -> np.ndarray:
    """
    Returns:
        np.ndarray: The tensor resulting from the contraction::

                     _____
               2____|     |
                    |  B  |
                    |_____|
                       |
                       |
                     __|__
               0____|     |
                    |  A  |
                    |_____|
                       |
                       |1
    """
    node1 = state1.nodes[node_id]
    node2 = state2.nodes[node_id]
    assert node1.is_leaf() and node2.is_leaf()
    errstr = "The leaf nodes must have exactly one open leg."
    assert len(node1.open_legs) == 2 and len(node2.open_legs) == 1, errstr
    tensor = np.tensordot(state2.tensors[node_id],state1.tensors[node_id],
                            axes=(node2.open_legs[0], node1.open_legs[1]))
    return tensor

In [204]:
ttn1 = ptn.random_big_ttns_two_root_children(ptn.RandomTTNSMode.DIFFVIRT)
ttn2 = ptn.random_big_ttns_two_root_children(ptn.RandomTTNSMode.DIFFVIRT)

In [205]:
possible_operators = ["X", "Y", "Z"]
H = ptn.random_hamiltonian(num_of_terms= 7,
                           possible_operators= possible_operators,
                           tree= ttn1,
                           strength = (-2,8),
                           num_sites = (5,6),
                           conversion_dict= {"X": X, "Y": Y, "Z": Z , "I2": np.eye(2), "I3": np.eye(3) , "I4": np.eye(4) , "I5": np.eye(5) , "I6": np.eye(6) , "I7": np.eye(7)})
H = H.pad_with_identities(ttn1 , symbolic= True)

ttno = ptn.TTNO.from_hamiltonian(H, ttn1)

In [206]:
v

NameError: name 'v' is not defined

In [240]:
from pytreenet.contractions.contraction_util import determine_index_with_ignored_leg

dictionary = ptn.PartialTreeCachDict()
computation_order = ttn1.linearise() # Getting a linear list of all identifiers

state1 = ttno 
state2  = ttn1

In [241]:
computation_order

['site2', 'site4', 'site5', 'site3', 'site1', 'site7', 'site6', 'site0']

In [242]:
node_id = 'site2'
parent_id = 'site1'
block = contract_leafs_2(node_id,ttno,ttn1)
dictionary.add_entry(node_id,parent_id,block)

node_id = 'site4'
parent_id = 'site3'
block = contract_leafs_2(node_id,ttno,ttn1)
dictionary.add_entry(node_id,parent_id,block)

node_id = 'site5'
parent_id = 'site3'
block = contract_leafs_2(node_id,ttno,ttn1)
dictionary.add_entry(node_id,parent_id,block)

In [243]:
# contract_subtrees_using_dictionary(node_id, next_node_id, state1,state2, dictionary)
node_id = 'site3'
next_node_id = 'site1'

In [244]:
ket_node, ket_tensor = state2[node_id]


In [245]:
ketblock_tensor = contract_all_but_one_neighbour_block_to_ket(ket_tensor,
                                                                ket_node,
                                                                next_node_id,
                                                                dictionary)

ketblock_tensor.shape

(5, 2, 4, 2, 4, 2)

In [246]:
bra_node, bra_tensor = state1[node_id]
bra_tensor.shape

(7, 4, 4, 2, 2)

In [247]:
list = []
for neighbour_id in ket_node.neighbouring_nodes():
    if neighbour_id != next_node_id:
        list.append(neighbour_id)
A = np.tensordot(ketblock_tensor,bra_tensor, axes=([bra_node.neighbour_index(a) * 2 for a in list] + [1],[bra_node.neighbour_index(a) for a in list] + [-1] ))
A = A.transpose([0,3,1,2,4])
A.shape

(5, 7, 2, 2, 2)

In [248]:
dictionary.add_entry('site3','site1',A)

In [249]:
node_id = 'site1'
next_node_id = 'site0'


In [250]:
ket_node, ket_tensor = state2[node_id]

In [251]:
ketblock_tensor = contract_all_but_one_neighbour_block_to_ket(ket_tensor,
                                                                ket_node,
                                                                next_node_id,
                                                                dictionary)

ketblock_tensor.shape

(7, 2, 2, 2, 7, 2, 2, 2)

In [254]:
bra_node, bra_tensor = state1[node_id]
bra_tensor.shape

(6, 2, 7, 2, 2)

In [255]:
list = []
for neighbour_id in ket_node.neighbouring_nodes():
    if neighbour_id != next_node_id:
        list.append(neighbour_id)
A = np.tensordot(ketblock_tensor,bra_tensor, axes=([bra_node.neighbour_index(a) * 2 for a in list] + [1],[bra_node.neighbour_index(a) for a in list] + [-1] ))


ValueError: axes don't match array

In [252]:
v

NameError: name 'v' is not defined

In [84]:
dictionary = ptn.PartialTreeCachDict()
computation_order = ttn1.linearise() # Getting a linear list of all identifiers
computation_order

state1 = ttn1 
state2  = ttn2

In [85]:
node_id = 'site2'
parent_id = 'site1'
block = contract_leafs(node_id,state1,state2)
dictionary.add_entry(node_id,parent_id,block)

node_id = 'site4'
parent_id = 'site3'
block = contract_leafs(node_id,state1,state2)
dictionary.add_entry(node_id,parent_id,block)

node_id = 'site5'
parent_id = 'site3'
block = contract_leafs(node_id,state1,state2)
dictionary.add_entry(node_id,parent_id,block)

In [86]:
dictionary[('site2', 'site1')].shape , dictionary[('site4', 'site3')].shape , dictionary[('site5', 'site3')].shape

((4, 4), (2, 2), (3, 3))

In [87]:
# contract_subtrees_using_dictionary(node_id, next_node_id, state1,state2, dictionary)
node_id = 'site3'
next_node_id = 'site1'


In [88]:
ket_node, ket_tensor = state1[node_id]
ketblock_tensor = contract_all_but_one_neighbour_block_to_ket(ket_tensor,
                                                                ket_node,
                                                                next_node_id,
                                                                dictionary)

In [89]:
ketblock_tensor.shape

(5, 2, 2, 3)

In [90]:
from pytreenet.contractions.contraction_util import determine_index_with_ignored_leg , contract_neighbour_block_to_ket_ignore_one_leg
neighbour_id = "site4"
ignoring_node_id = "site1"
determine_index_with_ignored_leg(ket_node,
                                                                 neighbour_id,
                                                                 ignoring_node_id)

1

In [ ]:
contract_neighbour_block_to_ket_ignore_one_leg(ket_tensor,
                                                   ket_node,
                                                   neighbour_id,
                                                   ignoring_node_id,
                                                   dictionary)